In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local[1]").appName('').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/09 13:41:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.option("inferSchema", "true").parquet("./ref/parquet")
df.createOrReplaceTempView("tb")

dicionario = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema", "true").option("encoding", "utf-8").csv("./dicionario_deter_sie.csv")
dicionario.createOrReplaceTempView("dicionario")

In [18]:
df_join = spark.sql("""

SELECT
tb.ano AS `ano`,
tb.empresa AS `empresa`,
prefixo,
sequencial,
CONCAT(ori_municipio_nome, '-', des_municipio_nome) as linha_ori_des_sie,
CONCAT(loc_ori.SIG_CONCAT_UPPER, '-', loc_des.SIG_CONCAT_UPPER) AS localidade_ori_des_sig,
km,
pax_total AS `passageiros_total`
FROM tb
LEFT JOIN dicionario AS loc_ori ON tb.ori_localidade_nome = loc_ori.SIE_NOME
LEFT JOIN dicionario AS loc_des ON tb.des_localidade_nome = loc_des.SIE_NOME
LEFT JOIN dicionario AS mun_ori ON tb.ori_municipio_nome = mun_ori.SIE_NOME
LEFT JOIN dicionario AS mun_des ON tb.des_municipio_nome = mun_des.SIE_NOME

""").show(10, False)

# WHERE ano = '2000' AND (pax_total > 0) AND (loc_ori.SIG_CONCAT_UPPER LIKE 'GUATAMBÚSC' OR loc_des.SIG_CONCAT_UPPER LIKE 'GUATAMBÚSC')

# WHERE ano = '2000' AND (pax_total > 0) AND (loc_ori.SIG_CONCAT_UPPER LIKE 'CHAPECÓSC' OR loc_des.SIG_CONCAT_UPPER LIKE 'CHAPECÓSC')

# df_join.write.option("delimiter", ",").option("header", "true").option("encoding", "utf-8").csv("./testecsv/")

# df_join.write.partitionBy("empresa").parquet("./test_joined")

+----+-----------------------------+-------+----------+-----------------------------+-----------------------------------------+------+-----------------+
|ano |empresa                      |prefixo|sequencial|linha_ori_des_sie            |localidade_ori_des_sig                   |km    |passageiros_total|
+----+-----------------------------+-------+----------+-----------------------------+-----------------------------------------+------+-----------------+
|2008|1-EMP STO ANJO DA GUARDA LTDA|16- 0  |1         |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-ITAJAÍSC            | 11,00|10140            |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16- 0  |2         |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-NAVEGANTESSC        | 26,00|173              |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16- 0  |3         |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-BALNEÁRIO PIÇARRASSC| 36,00| 3673            |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16- 0  |4         |BALNEARIO CAMBORIU-JOINVILL

In [19]:
spark.sql("""

SELECT ano AS Ano, empresa AS Empresa, CONCAT(ori_municipio_nome, '-', des_municipio_nome) AS Linha, ori_des_localidade_nome AS `Origem-destino`, pax_total AS `Total de passageiros`
FROM tb
WHERE ano = 2019
AND (empresa LIKE '23%' OR empresa LIKE '1505%')
AND pax_total > 10000
AND servico_ambito = 'RODOVIARIO'
ORDER BY pax_total DESC

""").show(10, False)

+----+---------------------------------+-----------------------+---------------------+--------------------+
|Ano |Empresa                          |Linha                  |Origem-destino       |Total de passageiros|
+----+---------------------------------+-----------------------+---------------------+--------------------+
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |CONCORDIA -CHAPECO     |CONCORDIA-CHAPECO    |18585               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |LAGES -FLORIANOPOLIS   |LAGES-FLORIANOPOLIS  |14683               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |FLORIANOPOLIS -BLUMENAU|BRUSQUE-BLUMENAU     |13541               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |CONCORDIA -CHAPECO     |SEARA-CHAPECO        |12997               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |LAGES -CURITIBANOS     |LAGES-CORREIA PINTO  |11451               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |MAFRA -JOINVILLE       |MAFRA-S BENTO DO SUL |11017               |
+----+----------------------